In [32]:
from sentinelhub import SentinelHubCatalog, BBox, Geometry, SHConfig, CRS, DataCollection
import geopandas as gpd
import os
import glob
import pandas as pd

In [100]:
def search_products(bbox, time_user):
    # Credentials
    config = SHConfig()
    config.sh_client_id = 'sh-dce21bb4-8cfb-48df-8f32-8c655d0a83f1'
    config.sh_client_secret = '6ZiLKeRpeyTNfrovxYLVgnMN7qj0HUv1'
    config.sh_base_url = 'https://sh.dataspace.copernicus.eu'
    config.sh_token_url = 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token'

    catalog = catalog = SentinelHubCatalog(config=config)

    #bbox = BBox(bbox=[12.44693, 41.870072, 12.541001, 41.917096], crs=CRS.WGS84)

    search_iterator = catalog.search(
      DataCollection.SENTINEL2_L2A,  
      bbox=bbox,
      #time=('2023-04-11', '2023-05-11'),
      time = time_user,
    )

    results = list(search_iterator)
    return results

In [101]:
def extract_bbox_from_shp(shapefile):
    gdf = gpd.read_file(shapefile)
    minx = gdf.geometry.bounds.minx
    miny = gdf.geometry.bounds.miny
    maxx = gdf.geometry.bounds.maxx
    maxy = gdf.geometry.bounds.maxy
    bbox = BBox(bbox = [minx, miny, maxx, maxy], crs=CRS.WGS84)
    return bbox

In [102]:
def find_shp_files(directory):
    # Create a list to store the paths
    shp_files = []
    # Use os.walk to traverse the directory
    for root, dirs, files in os.walk(directory):
        # Use glob to find all .shp files in the current directory
        shp_files.extend(glob.glob(os.path.join(root, '*.shp')))
    return shp_files

In [96]:
shp_files = find_shp_files('/home/sg/Projects/EMS/ems_zambia/2_AWARE_Shapefiles/CPM_shp_files')

In [97]:
time_user = ('2023-10-01', '2024-06-13')

In [105]:
# Define the start and end dates
start_date = '2023-10-01'
end_date = '2024-06-13'

# Generate a date range with a frequency of 2 days
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Loop through the date range and create pairs of dates
date_pairs = [(date_range[i], date_range[i + 1]) for i in range(0, len(date_range) - 1, 2)]

In [109]:
date_pairs[0][0].strftime('%Y-%m-%d'),date_pairs[0][1].strftime('%Y-%m-%d')

('2023-10-01', '2023-10-02')

In [128]:
products_final = pd.DataFrame()
for i, shapefile in enumerate(shp_files):
    for date in date_pairs:
        time_user = date[0].strftime('%Y-%m-%d'),date[1].strftime('%Y-%m-%d')
        bbox = extract_bbox_from_shp(shapefile)
        results = search_products(bbox, time_user)
        if results == []:
            continue
        products = pd.DataFrame(results)
        products['cloud'] = products['properties'].apply(lambda x: x['eo:cloud_cover'])
        products['aoi'] = shapefile.split('/')[-1]
        products = products[['id', 'properties', 'cloud', 'aoi']]
        if i == 0:
            products_final = products
        else:
            products_final = pd.concat([products_final, products])

In [132]:
products_final

,id,properties,cloud,aoi
0,S2A_MSIL2A_20240610T080611_N0510_R078_T35KNB_2...,"{'datetime': '2024-06-10T08:34:57.031Z', 'plat...",0.00,Malindi site.shp
0,S2A_MSIL2A_20231004T080801_N0509_R078_T35LND_2...,"{'datetime': '2023-10-04T08:34:23.234Z', 'plat...",2.56,Chikupe site.shp
0,S2B_MSIL2A_20231009T080809_N0509_R078_T35LND_2...,"{'datetime': '2023-10-09T08:34:23.012Z', 'plat...",0.00,Chikupe site.shp
0,S2A_MSIL2A_20231014T080901_N0509_R078_T35LND_2...,"{'datetime': '2023-10-14T08:34:20.1Z', 'platfo...",12.97,Chikupe site.shp
0,S2B_MSIL2A_20231019T080939_N0509_R078_T35LND_2...,"{'datetime': '2023-10-19T08:34:23.943Z', 'plat...",12.93,Chikupe site.shp
...,...,...,...,...
0,S2B_MSIL2A_20240602T075609_N0510_R035_T35LNC_2...,"{'datetime': '2024-06-02T08:24:42.207Z', 'plat...",15.43,Namagololo site.shp
0,S2B_MSIL2A_20240605T080609_N0510_R078_T35LNC_2...,"{'datetime': '2024-06-05T08:34:38.616Z', 'plat...",0.00,Namagololo site.shp
0,S2A_MSIL2A_20240607T075611_N0510_R035_T35LNC_2...,"{'datetime': '2024-06-07T08:24:46.494Z', 'plat...",0.00,Namagololo site.shp
0,S2A_MSIL2A_20240610T080611_N0510_R078_T35LNC_2...,"{'datetime': '2024-06-10T08:34:42.282Z', 'plat...",0.00,Namagololo site.shp


In [135]:
products_final.to_csv('/home/sg/Projects/EMS/ems_zambia/2_AWARE_Shapefiles/sentinel2_products.csv')

In [136]:
products_final['property_key'] = products_final['properties'].apply(lambda x: list(x.keys())[0])
products_final['property_value'] = products_final['properties'].apply(lambda x: list(x.values())[0])

In [137]:
grouped_df = products_final.groupby(['id','property_key', 'property_value']).agg({'aoi':list, 'cloud':'max'}).reset_index()

In [139]:
grouped_df.to_csv('/home/sg/Projects/EMS/ems_zambia/2_AWARE_Shapefiles/sentinel2_products_group.csv')

In [123]:
products

[{'stac_version': '1.0.0',
  'stac_extensions': ['https://stac-extensions.github.io/eo/v1.0.0/schema.json',
   'https://stac-extensions.github.io/projection/v1.0.0/schema.json'],
  'id': 'S2B_MSIL2A_20240612T075609_N0510_R035_T35LNC_20240612T111934.SAFE',
  'type': 'Feature',
  'geometry': {'type': 'MultiPolygon',
   'crs': {'type': 'name',
    'properties': {'name': 'urn:ogc:def:crs:OGC::CRS84'}},
   'coordinates': [[[[27.63505312742995, -15.37570073470031],
      [28.0228829399003, -15.37425715443158],
      [28.02790199536011, -16.36672815842601],
      [27.397532082884734, -16.368861597822935],
      [27.63505312742995, -15.37570073470031]]]]},
  'bbox': [27.397532082884734,
   -16.368861597822935,
   28.02790199536011,
   -15.37425715443158],
  'properties': {'datetime': '2024-06-12T08:24:44.075Z',
   'platform': 'sentinel-2b',
   'instruments': ['msi'],
   'constellation': 'sentinel-2',
   'gsd': 10,
   'eo:cloud_cover': 32.06,
   'proj:epsg': 32735,
   'proj:bbox': [499980.0, 81

In [82]:
start_date = "2023-10-01"
end_date = "2023-11-30"
bb = [27.24826251360173,-16.63208191671156,27.27238531279168,-16.60522985746961]
data_collection = "SENTINEL-2"
aoi = f"POLYGON(({bb[0]} {bb[1]},{bb[2]} {bb[1]},{bb[2]} {bb[3]},{bb[0]} {bb[3]},{bb[0]} {bb[1]}))'"

In [83]:
import requests
json = requests.get(f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=Collection/Name eq '{data_collection}' and OData.CSC.Intersects(area=geography'SRID=4326;{aoi}) and ContentDate/Start gt {start_date}T00:00:00.000Z and ContentDate/Start lt {end_date}T00:00:00.000Z").json()

In [84]:
products = pd.DataFrame.from_dict(json['value'])

In [92]:
products[products.OriginDate == '2023-10-04T13:34:10.957Z']

,@odata.mediaContentType,Id,Name,ContentType,ContentLength,OriginDate,PublicationDate,ModificationDate,Online,EvictionDate,S3Path,Checksum,ContentDate,Footprint,GeoFootprint
0,application/octet-stream,00497c57-9456-4904-908c-2d919a345d3f,S2A_MSIL1C_20231004T080801_N0509_R078_T35KNB_2...,application/octet-stream,555767069,2023-10-04T13:34:10.957Z,2023-10-04T13:41:34.394Z,2023-10-04T13:41:44.133Z,True,,/eodata/Sentinel-2/MSI/L1C/2023/10/04/S2A_MSIL...,"[{'Value': '4cfd51e399865ad700ddefdc720493d6',...","{'Start': '2023-10-04T08:08:01.024Z', 'End': '...",geography'SRID=4326;POLYGON ((27.7284199145779...,"{'type': 'Polygon', 'coordinates': [[[27.72841..."


In [ ]:
19/10
24/10
29/10
03/11
08/11
-
-
-
28/11
-
08/12
18/12
23/12
01/04
06/04
11/04
16/04
26/04
01/05
06/05
26/05